# **Проектное задание  - анализ таблиц с аэропортами**

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display

In [ ]:
#pip install d:\Downloads\plotly-3.5.0-py2.py3-none-any.whl
import plotly.offline as py
import plotly.graph_objs as go

In [ ]:
import awoc

In [ ]:
# Исходные данные на странице
#
# https://openflights.org/data.html
# 
url_airport = 'https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat'
url_routers = 'https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat'

df_raw_airport = pd.read_csv(url_airport)
display(df_raw_airport.head(3))
df_raw_routers = pd.read_csv(url_routers)
display(df_raw_routers.head(3))

,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081689834590001,145.391998291,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
0,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
1,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
2,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports


,2B,410,AER,2965,KZN,2990,Unnamed: 6,0,CR2
0,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
2,2B,410,CEK,2968,KZN,2990,NaN,0,CR2


In [ ]:
columns_airport = ['Airport ID', 'Name', 'City',
                   'Country', 'IATA', 'ICAO', 'Latitude', 'Longitude', 'Altitude', 'Timezone',
                   'DST', 'Tz database time zone', 'Type', 'Source']

columns_routers = ['Airline', 'Airline ID', 'Source airport', 'Source airport ID', 
                   'Destination airport', 'Destination airport ID', 'Codeshare', 'Stops', 'Equipment']

df_raw_airport = pd.read_csv(url_airport, names = columns_airport)
display(df_raw_airport.head(3))
df_raw_routers = pd.read_csv(url_routers, names = columns_routers)
display(df_raw_routers.head(3))

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.08169,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.20708,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.82679,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports


,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2


# Задание 1

Добавить столбец <code>Continent</code> с данными, которые будут содержать название континента.

In [ ]:
my_world = awoc.AWOC()


In [ ]:
def country_to_continent(name):
    try:
        continent = my_world.get_country_continent_name(name)
    except NameError:
        continent = np.nan
    return continent


In [ ]:
df_airport = df_raw_airport.copy()
df_airport['Continent'] = df_airport['Country'].map(country_to_continent)
dict_country_error = {
    "Cote d'Ivoire": 'Ivory Coast',
    'Congo (Kinshasa)': 'Democratic Republic of the Congo',
    'Congo (Brazzaville)':'Republic of the Congo', 
    'Burma':'Myanmar',
    'Cocos (Keeling) Islands': 'Cocos Islands'
}
df_airport['Country'] = df_airport['Country'].replace(dict_country_error)
df_airport['Continent'] = df_airport['Country'].map(country_to_continent)
df_airport[df_airport['Continent'].isna()]
dict_teritor_error = {
    'West Bank': 'Asia', 
    'Midway Islands': 'Oceania', 
    'French Guiana': 'South America', 
    'Martinique': 'North America',
    'Guadeloupe': 'North America', 
    'Virgin Islands': 'North America', 
    'Norfolk Island': 'Oceania', 
    'Johnston Atoll': 'Oceania',
    'Svalbard': 'Europe', 
    'Wake Island': 'Oceania'
}
df_airport['Continent'] = df_airport['Continent'].fillna( df_airport['Country'].map(dict_teritor_error) )



In [ ]:
df_airport[df_airport['Continent'].isna()] # финальная проверка

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source,Continent


# Задание 2

Из списка континентов случайно выбрать континент и срезать таблицу по названию континента и аэропорту имеющему код IATA (имеется в виду - для выбранного континента только аэропорты с кодом IATA). Каждый раз при выполнении кода континент случайно выбирается.

In [ ]:
np.random.seed(42)
Continents = df_airport['Continent'].unique()
n = len(df_airport['Continent'].unique())  #количество континентов согласно данным из нашей таблицы
x = np.random.randint(0,(n-1))             # n-1 чтобы не вылетать за пределы массива из-за индексации питона
name = Continents[x]
df_continent = df_airport[(df_airport['IATA'] != "\\N") & (df_airport['Continent'] == name)] # есть код иата \
                                                                                             #+ срез по нашему континенту
df_continent


                      


,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source,Continent
205,207,Blida Airport,Blida,Algeria,QLD,DAAB,36.503601,2.814170,535,1,N,Africa/Algiers,airport,OurAirports,Africa
206,208,Bou Saada Airport,Bou Saada,Algeria,BUJ,DAAD,35.332500,4.206390,1506,1,N,Africa/Algiers,airport,OurAirports,Africa
207,209,Soummam Airport,Bejaja,Algeria,BJA,DAAE,36.712002,5.069920,20,1,N,Africa/Algiers,airport,OurAirports,Africa
208,210,Houari Boumediene Airport,Algier,Algeria,ALG,DAAG,36.691002,3.215410,82,1,N,Africa/Algiers,airport,OurAirports,Africa
209,211,Djanet Inedbirene Airport,Djanet,Algeria,DJG,DAAJ,24.292801,9.452440,3176,1,N,Africa/Algiers,airport,OurAirports,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7634,13714,"Paloich Airport, Heliport",Higleig,South Sudan,HGI,HSFA,10.529167,32.500556,1290,\N,\N,\N,airport,OurAirports,Africa
7650,13756,Arathusa Safari Lodge Airport,Arathusa,South Africa,ASS,FACC,-24.744165,31.522499,1200,2,U,\N,airport,OurAirports,Africa
7667,13808,Mchauru Airport,Geita,Tanzania,GIT,HTRU,-2.813667,32.172472,3955,3,N,\N,airport,OurAirports,Africa
7668,13821,Gitega Airport,Gitega,Burundi,GID,HBBE,-3.417209,29.911308,5741,2,U,\N,airport,OurAirports,Africa


# Задание 3

На основе таблицы связи аэропортов создать таблицу с кодом аэропорта, количеством взлетов и количеством посадок.

In [ ]:
df1 = df_raw_routers.groupby('Source airport').count()['Source airport ID']
df2 = df_raw_routers.groupby('Destination airport').count()['Destination airport ID']
df3 = pd.concat([df1,df2],axis=1) #соединил по колонкам
df3.rename(columns = {'Source airport ID': 'Departure_amount', 'Destination airport ID': 'Arrival_amount'}, inplace=True)
df3




,Departure_amount,Arrival_amount
AAE,9.0,9.0
AAL,20.0,21.0
AAN,2.0,2.0
AAQ,3.0,3.0
AAR,8.0,8.0
...,...,...
ORX,NaN,3.0
QFX,NaN,1.0
SPI,NaN,5.0
TUA,NaN,1.0


# Задание 4

Используя в качестве ключа код аэропорта слить данные для отобранного континента и собрать таблицу - страна, название аэропорта, количество взлет/посадок (сумма взлетов и посадок)

In [ ]:
df4 = df3.reset_index()
df4.rename(columns={'index': 'IATA'},inplace=True)           #перевел, чтобы можно было слить по IATA
df4['Sum'] = df4['Departure_amount'] + df4['Arrival_amount'] # вместо двух столбцов один с суммой взлетов и посадок
df5 = pd.merge(df_continent[['Country', 'Name', 'IATA']],df4[['IATA', 'Sum']] , on='IATA', how= 'left')
df5.drop('IATA',1)                                           # "чистая" таблица без иаты

,Country,Name,Sum
0,Algeria,Blida Airport,NaN
1,Algeria,Bou Saada Airport,NaN
2,Algeria,Soummam Airport,12.0
3,Algeria,Houari Boumediene Airport,176.0
4,Algeria,Djanet Inedbirene Airport,3.0
...,...,...,...
621,South Sudan,"Paloich Airport, Heliport",NaN
622,South Africa,Arathusa Safari Lodge Airport,NaN
623,Tanzania,Mchauru Airport,NaN
624,Burundi,Gitega Airport,NaN


# Задание 5

1. Найти коды аэропортов для которых нет взлет/посадок в таблице аэропортов.
2. Найти коды аэропортов для которых нет названий в таблице связей аэропортов.

In [ ]:
df6 = df_airport[(df_airport['IATA'] != "\\N")]                                 #есть коды иата из первой таблицы
df10 = df3[(df3['Departure_amount'].notna()) | (df3['Arrival_amount'].notna())] #есть код иата хотя бы в взлетах/посадках
df10.fillna(0,inplace = True)                                        #заполняю "нан" 0,чтобы найти "наны " при объединении
df11 = df10.reset_index()  
df11.rename(columns={'index': 'IATA'},inplace=True)    
df7 = pd.merge(df6[['Country', 'Name', 'IATA']],df11[['IATA', 'Departure_amount', 'Arrival_amount']] , on='IATA', how= 'left')
df12 = df7[df7['Departure_amount'].isna()]       #тк изначально во взлетах и посадках "нанов" не было, то наличие "нан"\                                             # в одном столбцов и будет показывать, что для данного аэропорта нет данных во 2 таблице
output_1 = df12['IATA'].tolist()
len(output_1)

2810

In [ ]:
df9 = pd.merge(df6[['Name', 'IATA']], df4[['IATA']], on='IATA', how ='right') 
df9_ = df9[df9['Name'].isna()]
#слил по тем и только тем кодам IATA,кот есть во второй таблице, "нан" в столбце имени аэропорта означают, что данного кода IATA нет в первой таблице
output_2 = df9_['IATA'].tolist()
len(output_2)

163

# Задание 6

Для таблицы в задании 4, для трех крупнейших аэропортов для каждой страны, собрать ряд - первый индекс: название страны, второй индекс: название аэропорта, значение: количество взлет посадок. Первый индекс упорядочен по названию страны (по алфавиту), а назание аэропорта по количеству взлет посадок от большего к меньшему.

In [ ]:
large_airports = df5.groupby(['Country','Name'])['Sum'].sum()
large_airports.groupby('Country').nlargest(3).reset_index(level=0, drop=True)


# Задание 7

Для каждого континента отобрать не более 3 стран с наибольшим количеством взлет посадок и нанести их на географическую карту. 

In [ ]:
large_airports2 = df5.groupby('Country')['Sum'].sum().nlargest(3)
large_airports2
countries1 = ['MAR', 'EGY', 'ZAF']
z = [1,2,2]
layout = dict(title = 'Top 3 countries by total take-off and landing of aircrafts in Africa',
              geo={'scope': 'world'}) #словарь для карты
scl = [[0.0, 'rgb(44, 40, 82)'],[0.2, 'rgb(44, 40, 82)'],[0.4, 'rgb(44, 40, 82)'],\
            [0.6, 'rgb(44, 40, 82)'],[0.8, 'rgb(44, 40, 82)'],[1.0, 'rgb(44, 40, 82)']] #палитра

data = dict(                          #словарь для данных
    type='choropleth',
    locations=countries1,
    locationmode='ISO-3',
    colorscale=scl,
    autocolorscale = False,
    marker = dict(line = dict (color = 'rgb(255,255,255)', width = 2)),
    z=z)
map = go.Figure(data=[data], layout=layout)
py.plot(map)
#вывод в формате html -с другими варинатами вывода ошибку не смог исправить

# Задание 8

Для каждого континента отобрать 5 наиболее крупных аэропортов с наибольшим количеством взлет посадок

In [ ]:
airports_5 = df5.groupby(['Country','Name'])['Sum'].sum().nlargest(5).reset_index(level=0, drop=True)
airports_5


